In [ ]:
import random
import matplotlib 
matplotlib.use('Agg') 
% matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
## looking at the data

In [ ]:
from keras.datasets import cifar10
from keras.utils import np_utils
from keras.preprocessing.image import array_to_img


nb_classes = 10

# input image dimensions
img_rows, img_cols = 32, 32

# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
input_shape = (img_rows, img_cols, 3)

list_concepts = ['airplane', 
                 'automobile', 
                 'bird', 
                 'cat', 
                 'deer', 
                 'dog', 
                 'frog', 
                 'horse', 
                 'ship', 
                 'truck']

nb_classes = len(list_concepts)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

In [ ]:
i = random.randint(0, len(y_train))
print i, 
print y_train[i], list_concepts[y_train[i][0]]
array_to_img(X_train[i])

In [ ]:
## train the network

In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Dense, Dropout, Activation, Flatten

In [ ]:
## magic numbers:
batch_size = 128
nb_epoch = 40

# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)

model = Sequential()
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))

model.add(MaxPooling2D(pool_size=pool_size))

model.add(Flatten())
model.add(Dense(128))
model.add(Dense(nb_classes))
model.add(Activation('sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer=SGD(),
              metrics=['accuracy'])

plot_history_callback = PlotHistory('cifar10.png')
save_snapshots = ModelCheckpoint('cifar10.h5')

history = model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=2, validation_data=(X_test, Y_test), 
                    callbacks=[plot_history_callback, save_snapshots])
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

In [ ]:
from keras.callbacks import History, ModelCheckpoint

class PlotHistory(History):
    def __init__(self, file_name='history.png'):
        History.__init__(self)
        self.file_name = file_name
    def on_epoch_end(self, epoch, logs={}):
        History.on_epoch_end(self, epoch, logs)
        self.plot_logs()
    def plot_logs(self):
        evaluation_cost = self.history['val_loss']
        evaluation_accuracy = self.history['val_acc']
        training_cost = self.history['loss']
        training_accuracy = self.history['acc']
        f, (ax1, ax2) = plt.subplots(1, 2)
        f.set_figwidth(10)
        ax1.plot(evaluation_cost,label= 'test')
        ax1.plot(training_cost, label='train')
        ax1.set_title('Cost')
        ax1.legend()
        ax2.plot(evaluation_accuracy, label='test')
        ax2.plot(training_accuracy, label='train')
        ax2.set_title('Accuracy')
        ax2.legend(loc='lower right')
        f.savefig(self.file_name)
        plt.close(f)

In [ ]:
## looking at our model

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

from PIL import Image
import requests
import StringIO
import numpy
def load_photo_from_url(image_url, target_size=None, time_out_image_downloading=1):
    response = requests.get(image_url, timeout=time_out_image_downloading)
    image = Image.open(StringIO.StringIO(response.content))
    image = image.resize((target_size[1], target_size[0]))
    img_array = img_to_array(image) / 255.0
    img_array = img_array.reshape((1,) + img_array.shape)
    return img_array

In [ ]:
url = 'https://cdn.eyeem.com/thumb/eccf1bed261702dc7fcccc283fdb922031827c15-1447435613631/w/1250'
predictions = model.predict(load_photo_from_url(url,(32,32)))
list_concepts[numpy.argmax(predictions)]